In [ ]:
# STEP 1: Compare the raw sample file we received from the field (BizeGelen_Ham) with the raw sample file from Mat (Mat_Ham),
# and also perform internal checks within each file.

In [ ]:
import pandas as pd
import numpy as np

# File paths
bizegelen_path = r"C:/Users/kubra/Desktop/proje/Raw_Sample_field.xlsx"
mat_path = r"C:/Users/kubra/Desktop/proje/Raw_Sample_lab.xlsx"

# Columns to be used
columns = ["Numune İşareti", "Na2O", "K2O", "Cl", "Na"]

# Read the files
bizegelen = pd.read_excel(bizegelen_path, usecols=columns)
mat = pd.read_excel(mat_path, usecols=columns)

# Type conversions
bizegelen["Numune İşareti"] = bizegelen["Numune İşareti"].astype(str)
mat["Numune İşareti"] = mat["Numune İşareti"].astype(str)

for col in ["Na2O", "K2O", "Cl", "Na"]:
    bizegelen[col] = pd.to_numeric(bizegelen[col], errors="coerce")
    mat[col] = pd.to_numeric(mat[col], errors="coerce")

# Merge operation
merged = pd.merge(bizegelen, mat, on="Numune İşareti", how="outer", suffixes=('_BizeGelen', '_Mat'))

# 1 - Check rows where 'Numune İşareti' (Sample Code) exists but all chemical values (Na2O, K2O, Cl, Na) are missing

# 1.1 - Extract rows in BizeGelen_Ham where 'Numune İşareti' exists but there are no chemical analysis results at all;
# export these rows to a separate Excel sheet.

# 1.1 - Sample codes present in BizeGelen_Ham with no chemical results
ko1_1 = bizegelen[bizegelen[["Na2O", "K2O", "Cl", "Na"]].isna().all(axis=1)]

# 1.2 - Extract rows in Mat_Ham where 'Numune İşareti' exists but there are no chemical analysis results at all;
# export these rows to a separate Excel sheet.

# 1.2 - Same check for Mat_Ham
ko1_2 = mat[mat[["Na2O", "K2O", "Cl", "Na"]].isna().all(axis=1)]

# 1.3 - If there is data associated with a 'Numune İşareti' in BizeGelen_Ham but the same sample is not present in Mat_Ham,
# I want to see these unmatched rows in a separate Excel sheet.

# 1.3 - Records present in BizeGelen_Ham but missing in Mat_Ham
ko1_3 = merged[
    merged[["Na2O_BizeGelen", "K2O_BizeGelen", "Cl_BizeGelen", "Na_BizeGelen"]].notna().any(axis=1) &
    merged[["Na2O_Mat", "K2O_Mat", "Cl_Mat", "Na_Mat"]].isna().all(axis=1)
]

# 1.4 - If there is data associated with a 'Numune İşareti' in Mat_Ham but the same sample is not present in BizeGelen_Ham,
# I want to see these unmatched rows in a separate Excel sheet.

# 1.4 - The reverse case: Records present in Mat_Ham but missing in BizeGelen_Ham
ko1_4 = merged[
    merged[["Na2O_Mat", "K2O_Mat", "Cl_Mat", "Na_Mat"]].notna().any(axis=1) &
    merged[["Na2O_BizeGelen", "K2O_BizeGelen", "Cl_BizeGelen", "Na_BizeGelen"]].isna().all(axis=1)
]

# 1.5 - Extract rows where the same 'Numune İşareti' exists in both files, but there is no data in either;
# export this to a separate Excel sheet.

# 1.5 - Missing data in both files for the same sample codes
ko1_5 = merged[
    merged[["Na2O_BizeGelen", "K2O_BizeGelen", "Cl_BizeGelen", "Na_BizeGelen"]].isna().all(axis=1) &
    merged[["Na2O_Mat", "K2O_Mat", "Cl_Mat", "Na_Mat"]].isna().all(axis=1)
]

# 2 - Check rows where Na2O is missing but Na is present in the chemical analysis results for the same 'Numune İşareti'


bg_na2o_na = bizegelen[bizegelen['Na2O'].isna() & bizegelen['Na'].notna()]
mat_na2o_na = mat[mat['Na2O'].isna() & mat['Na'].notna()]

# 2.1 - Extract rows in BizeGelen_Ham where Na2O is missing but Na is available;
# export these rows to a separate Excel sheet.
ko2_1 = bg_na2o_na.copy()

# 2.2 - Provide me with a separate Excel sheet listing the rows in MAT_Ham where "Na2O" is missing
# but "Na" is present for a given "Sample Identifier".
ko2_2 = mat_na2o_na.copy()

# 2.3 - In the BizeGelen_Ham file, if a row with a given "Sample Identifier" contains "Na" but not "Na2O",
# and if that same "Sample Identifier" is not found in Mat_Ham, I want to see these unmatched rows.
# Please provide them in a separate Excel sheet.
ko2_3 = bg_na2o_na[~bg_na2o_na['Numune İşareti'].isin(mat['Numune İşareti'])]

# 2.4 - In the Mat_Ham file, if a row with a given "Sample Identifier" contains "Na" but not "Na2O",
# and if that same "Sample Identifier" is not found in BizeGelen_Ham, I want to see these unmatched rows.
# Please provide them in a separate Excel sheet.
ko2_4 = mat_na2o_na[~mat_na2o_na['Numune İşareti'].isin(bizegelen['Numune İşareti'])]

# 2.5 - List all rows with the same "Sample Identifier" where both in BizeGelen_Ham and MAT_Ham,
# "Na" is present but "Na2O" is missing. Provide these rows in a separate Excel sheet.
ko2_5 = pd.merge(bg_na2o_na, mat_na2o_na, on='Numune İşareti')

# 3 - Provide all rows where the value for "Na2O" is missing for the given Sample Identifier. 

# 3.1 - Provide all rows in BizeGelen_Ham where "Na2O" is missing for the given Sample Identifier
# in a separate Excel sheet.
ko3_1 = bizegelen[bizegelen['Na2O'].isna()]

# 3.2 - Provide all rows in MAT_Ham where "Na2O" is missing for the given Sample Identifier
# in a separate Excel sheet.
ko3_2 = mat[mat['Na2O'].isna()]

# 3.3 - If a "Sample Identifier" has a value for "Na2O" in BizeGelen_Ham but the same identifier
# lacks a "Na2O" value in Mat_Ham, list those rows in a separate Excel sheet.
ko3_3 = pd.merge(bizegelen[bizegelen['Na2O'].notna()], mat[mat['Na2O'].isna()], on='Numune İşareti')

# 3.4 - If a "Sample Identifier" has a value for "Na2O" in Mat_Ham but the same identifier
# lacks a "Na2O" value in BizeGelen_Ham, list those rows in a separate Excel sheet.
ko3_4 = pd.merge(mat[mat['Na2O'].notna()], bizegelen[bizegelen['Na2O'].isna()], on='Numune İşareti')

# 3.5 - Provide all rows with the same "Sample Identifier" where both in BizeGelen_Ham and MAT_Ham,
# "Na2O" is missing. Provide these rows in a separate Excel sheet.
ko3_5 = pd.merge(ko3_1, ko3_2, on='Numune İşareti')

# 4 - Provide all rows where the value for "K2O" is missing for the given Sample Identifier.

# 4.1 - Provide all rows in BizeGelen_Ham where "K2O" is missing for the given Sample Identifier
# in a separate Excel sheet.
ko4_1 = bizegelen[bizegelen['K2O'].isna()]

# 4.2 - Provide all rows in MAT_Ham where "K2O" is missing for the given Sample Identifier
# in a separate Excel sheet.
ko4_2 = mat[mat['K2O'].isna()]

# 4.3 - If a "Sample Identifier" has a value for "K2O" in BizeGelen_Ham but the same identifier
# lacks a "K2O" value in Mat_Ham, list those rows in a separate Excel sheet.
ko4_3 = pd.merge(bizegelen[bizegelen['K2O'].notna()], mat[mat['K2O'].isna()], on='Numune İşareti')

# 4.4 - If a "Sample Identifier" has a value for "K2O" in Mat_Ham but the same identifier
# lacks a "K2O" value in BizeGelen_Ham, list those rows in a separate Excel sheet.
ko4_4 = pd.merge(mat[mat['K2O'].notna()], bizegelen[bizegelen['K2O'].isna()], on='Numune İşareti')

# 4.5 - Provide all rows with the same "Sample Identifier" where both in BizeGelen_Ham and MAT_Ham,
# "K2O" is missing. Provide these rows in a separate Excel sheet.
ko4_5 = pd.merge(ko4_1, ko4_2, on='Numune İşareti')

# 5 - When both "Na2O" and "Na" are missing

# 5.1 - In BizeGelen_Ham, provide the rows where both "Na2O" and "Na" are missing for the given Sample Identifier.
# Present them in a separate Excel sheet.
ko5_1 = bizegelen[bizegelen['Na2O'].isna() & bizegelen['Na'].isna()]

# 5.2 - In Mat_Ham, provide the rows where both "Na2O" and "Na" are missing for the given Sample Identifier.
# Present them in a separate Excel sheet.
ko5_2 = mat[mat['Na2O'].isna() & mat['Na'].isna()]

# 5.3 - If both "Na2O" and "Na" are present in BizeGelen_Ham but not in Mat_Ham for the same Sample Identifier,
# list those missing rows in a separate Excel sheet.
ko5_3 = pd.merge(
    bizegelen[bizegelen['Na2O'].notna() & bizegelen['Na'].notna()],
    mat[mat['Na2O'].isna() & mat['Na'].isna()],
    on='Numune İşareti'
)

# 5.4 - If both "Na2O" and "Na" are present in Mat_Ham but not in BizeGelen_Ham for the same Sample Identifier,
# list those missing rows in a separate Excel sheet.
ko5_4 = pd.merge(
    mat[mat['Na2O'].notna() & mat['Na'].notna()],
    bizegelen[bizegelen['Na2O'].isna() & bizegelen['Na'].isna()],
    on='Numune İşareti'
)

# 5.5 - List all rows where both "Na2O" and "Na" are missing in both BizeGelen_Ham and Mat_Ham
# for the same Sample Identifier in a separate Excel sheet.
ko5_5 = pd.merge(ko5_1, ko5_2, on='Numune İşareti')

# 6 - Provide all rows where the value for "Cl" is missing for the given Sample Identifier.

# 6.1 - In BizeGelen_Ham, provide the rows where "Cl" is missing for the given Sample Identifier
# in a separate Excel sheet.
ko6_1 = bizegelen[bizegelen['Cl'].isna()]

# 6.2 - In Mat_Ham, provide the rows where "Cl" is missing for the given Sample Identifier
# in a separate Excel sheet.
ko6_2 = mat[mat['Cl'].isna()]

# 6.3 - If "Cl" is present in BizeGelen_Ham but missing in Mat_Ham for the same Sample Identifier,
# provide these Sample Identifiers in a separate Excel sheet.
ko6_3 = pd.merge(bizegelen[bizegelen['Cl'].notna()], mat[mat['Cl'].isna()], on='Numune İşareti')

# 6.4 - If "Cl" is present in Mat_Ham but missing in BizeGelen_Ham for the same Sample Identifier,
# provide these Sample Identifiers in a separate Excel sheet.
ko6_4 = pd.merge(mat[mat['Cl'].notna()], bizegelen[bizegelen['Cl'].isna()], on='Numune İşareti')

# 6.5 - Provide all rows with the same Sample Identifier where "Cl" is missing in both
# BizeGelen_Ham and MAT_Ham, in a separate Excel sheet.
ko6_5 = pd.merge(ko6_1, ko6_2, on='Numune İşareti')

# Write results to Excel
output_path = r"C:/Users/kubra/Desktop/proje/hamorneklem.xlsx"
with pd.ExcelWriter(output_path) as writer:
    ko1_1.to_excel(writer, sheet_name="1.1_BizeGelen_Bos", index=False)
    ko1_2.to_excel(writer, sheet_name="1.2_MatHam_Bos", index=False)
    ko1_3.to_excel(writer, sheet_name="1.3_MatHam_eksik", index=False)
    ko1_4.to_excel(writer, sheet_name="1.4_BizeGelen_eksik", index=False)
    ko1_5.to_excel(writer, sheet_name="1.5_Herikisi_Bos", index=False)

    ko2_1.to_excel(writer, sheet_name="2.1_BG_NaVar_Na2OYok", index=False)
    ko2_2.to_excel(writer, sheet_name="2.2_MAT_NaVar_Na2OYok", index=False)
    ko2_3.to_excel(writer, sheet_name="2.3_BGdeVar_MATteYok", index=False)
    ko2_4.to_excel(writer, sheet_name="2.4_MATteVar_BGdeYok", index=False)
    ko2_5.to_excel(writer, sheet_name="2.5_Herİkisi", index=False)

    ko3_1.to_excel(writer, sheet_name="3.1_BG_Na2OYok", index=False)
    ko3_2.to_excel(writer, sheet_name="3.2_MAT_Na2OYok", index=False)
    ko3_3.to_excel(writer, sheet_name="3.3_BGvar_MATyok", index=False)
    ko3_4.to_excel(writer, sheet_name="3.4_MATvar_BGyok", index=False)
    ko3_5.to_excel(writer, sheet_name="3.5_HerikisiNa2OYok", index=False)

    ko4_1.to_excel(writer, sheet_name="4.1_BG_K2OYok", index=False)
    ko4_2.to_excel(writer, sheet_name="4.2_MAT_K2OYok", index=False)
    ko4_3.to_excel(writer, sheet_name="4.3_BGvar_MATyok", index=False)
    ko4_4.to_excel(writer, sheet_name="4.4_MATvar_BGyok", index=False)
    ko4_5.to_excel(writer, sheet_name="4.5_HerikisiK2OYok", index=False)

    ko5_1.to_excel(writer, sheet_name="5.1_BG_Na_Na2OYok", index=False)
    ko5_2.to_excel(writer, sheet_name="5.2_MAT_Na_Na2OYok", index=False)
    ko5_3.to_excel(writer, sheet_name="5.3_BGvar_MATyok", index=False)
    ko5_4.to_excel(writer, sheet_name="5.4_MATvar_BGyok", index=False)
    ko5_5.to_excel(writer, sheet_name="5.5_HerikisiNaNa2OYok", index=False)

    ko6_1.to_excel(writer, sheet_name="6.1_BG_CLYok", index=False)
    ko6_2.to_excel(writer, sheet_name="6.2_MAT_CLYok", index=False)
    ko6_3.to_excel(writer, sheet_name="6.3_BGvar_MATyok", index=False)
    ko6_4.to_excel(writer, sheet_name="6.4_MATvar_BGyok", index=False)
    ko6_5.to_excel(writer, sheet_name="6.5_HerikisiCLYok", index=False)

print("✅ Operations completed successfully and written to Excel file:")
print(output_path)

In [ ]:
# STEP 2: Compare the Excel sheet obtained from Step 1, Condition 1.4 with the KK-KG data table
# Condition 1.4 refers to the Excel file showing the Sample Identifiers that exist in the Mat_Ham file
# but do not exist in the BizeGelen_Ham file.

In [ ]:
import pandas as pd

# File paths
kosul_1_4_path = r"C:/Users/kubra/Desktop/proje/hamorneklem.xlsx"
kk_kg_veri_kontrolu_path = r"C:/Users/kubra/Desktop/proje/hamorneklem.xlsx"
output_path = r"C:/Users/kubra/Desktop/proje/hamorneklem.xlsx"

# Read the files
kosul_1_4 = pd.read_excel(kosul_1_4_path)
kk_kg_veri_kontrolu = pd.read_excel(kk_kg_veri_kontrolu_path)

# Convert the "Sample Identifier" columns to string type
kosul_1_4["Numune İşareti"] = kosul_1_4["Numune İşareti"].astype(str)
kk_kg_veri_kontrolu["Numune İşareti"] = kk_kg_veri_kontrolu["Numune İşareti"].astype(str)

# Condition 1 - Sample Identifiers present in kosul_1_4 but not in kk_kg_veri_kontrolu
adim2_kos1 = kosul_1_4[~kosul_1_4["Numune İşareti"].isin(kk_kg_veri_kontrolu["Numune İşareti"])]

# Condition 2 - Sample Identifiers present in kk_kg_veri_kontrolu but not in kosul_1_4
adim2_kos2 = kk_kg_veri_kontrolu[~kk_kg_veri_kontrolu["Numune İşareti"].isin(kosul_1_4["Numune İşareti"])]

# Condition 3 - Sample Identifiers common to both files
common_ids = set(kosul_1_4["Numune İşareti"]) & set(kk_kg_veri_kontrolu["Numune İşareti"])
adim2_kos3_kosul_1_4 = kosul_1_4[kosul_1_4["Numune İşareti"].isin(common_ids)]
adim2_kos3_kk_kg = kk_kg_veri_kontrolu[kk_kg_veri_kontrolu["Numune İşareti"].isin(common_ids)]

# Are all the additionally found sample names present in KK_KG?
if set(kosul_1_4["Numune İşareti"]).issubset(set(kk_kg_veri_kontrolu["Numune İşareti"])):
    mesaj = "Fazladan bulduğunuz numune isimlerinin tamamı KK_KG örnekleridir."
else:
    mesaj = "Fazladan bulunan numune isimlerinin bazıları KK_KG dosyasında bulunmamaktadır."

# Write to Excel file
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    adim2_kos1.to_excel(writer, sheet_name="Adim2_Kosul1_Yok_KK_KG", index=False)
    adim2_kos2.to_excel(writer, sheet_name="Adim2_Kosul2_Yok_Kosul_1_4", index=False)
    adim2_kos3_kosul_1_4.to_excel(writer, sheet_name="Adim2_Kosul3_Kosul_1_4", index=False)
    adim2_kos3_kk_kg.to_excel(writer, sheet_name="Adim2_Kosul3_KK_KG", index=False)
    # Mesajı ayrı bir sayfaya yaz
    df_mesaj = pd.DataFrame([{"Sonuç": mesaj}])
    df_mesaj.to_excel(writer, sheet_name="Adim2_Sonuc", index=False)

print("✅ Operations completed successfully and written to Excel file:")
print(output_path)
